In [1]:
!pip install -U selenium
!pip3 install webdriver-manager
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys




Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The 
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for 
guitars.

In [17]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

In [18]:
url = "https://www.amazon.in/"
driver.get(url)

In [19]:
user_input = input('Enter the product that we want to search : ')

Enter the product that we want to search : guitar


In [20]:
search = driver.find_element(By.ID,"twotabsearchtextbox")
search

search.send_keys(user_input)

search_btn = driver.find_element(By.XPATH,"//div[@class='nav-search-submit nav-sprite']/span/input")

search_btn.click()

Q2.In the above question, now scrape the following details of each product listed in first 3 pages of your search 
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then 
scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [21]:
urls = []    
for i in range(0,3):   
    page_url = driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for i in page_url:
        urls.append(i.get_attribute("href"))
next_btn = driver.find_elements(By.XPATH,"//li[@class='a-last']/a")
time.sleep(3)

In [22]:
len(urls)

183

In [23]:
Brand_Name = [] 
Product_Name = [] 
Price = [] 
Exchange = [] 
Expected_delivery = [] 
Availability = []

In [24]:
for i in urls:
    driver.get(i)
    time.sleep(3)
    
    try:
        brand = driver.find_element(By.XPATH,"//div[@class='a-section a-spacing-small a-spacing-top-small']/table/tbody/tr/td[2]")
        Brand_Name.append(brand.text)
    except NoSuchElementException:
        Brand_Name.append('-')
    try:
        name = driver.find_element(By.XPATH,"//span[@id='productTitle']")
        Product_Name.append(name.text)
    except NoSuchElementException:
        Product_Name.append('-')
    try:
        price = driver.find_element(By.XPATH,"//span[@class='a-price aok-align-center reinventPricePriceToPayMargin priceToPay']")
        Price.append(price.text)
    except NoSuchElementException:
        Price.append('-')
    try:
        Return = driver.find_element(By.XPATH,"//li[3][@class='a-carousel-card tw-scroll-carousel-element']/div/span/div[2]/span")
        Exchange.append(Return.text)
    except NoSuchElementException:
        Exchange.append('-')    
    try:
        exp_del = driver.find_element(By.XPATH,"//div[1][@class='a-spacing-base']/span/span")
        Expected_delivery.append(exp_del.text)
    except NoSuchElementException:
        Expected_delivery.append('-')
    try:
        availability = driver.find_element(By.XPATH,"//span[@class='a-size-medium a-color-success']")
        Availability.append(availability.text)
    except NoSuchElementException:
        Availability.append('-')

In [25]:
print(len(Brand_Name),
      len(Product_Name),
      len(Price),
      len(Exchange),
      len(Expected_delivery),
      len(Availability))

183 183 183 183 183 183


In [26]:
Product = pd.DataFrame({})
Product["Brand Name"] = Brand_Name
Product["Product Name"] = Product_Name
Product["Price"] = Price
Product["Return/Exchange"] = Exchange
Product["Expected Delivery"] = Expected_delivery
Product["Availability"] = Availability
Product["Product URL"] = urls
Product

,Brand Name,Product Name,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence,Kadence Frontier guitar with Online Guitar lea...,"₹4,999",7 days Replacement,"Wednesday, 14 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,DEVICE OF URBAN INFOTECH,DEVICE OF URBAN INFOTECH 41 inch Acoustic Guit...,"₹5,448",7 days Returnable,"Saturday, 17 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Henrix,Henrix 38C 38 Inch Cutaway Acoustic Guitar Wit...,"₹2,999",7 days Replacement,"Thursday, 15 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,VAULT,Vault DA40 41 Inch Premium Solid Spruce-Top Dr...,"₹6,934",7 days Replacement,"Saturday, 17 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,JUAREZ,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹1,999",7 days Replacement,"Friday, 16 June",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
178,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,"₹6,899",7 days Replacement,"Friday, 16 June",In stock,https://www.amazon.in/Kadence-Frontier-Acousti...
179,IMAGINEA,Imaginea® 41 Inch Acoustic Guitar Semi Wood an...,"₹4,999",7 days Returnable,"Saturday, 17 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
180,Guitar Bro,GUITAR BRO - COMBO (Black Acoustic Guitar for ...,"₹8,999",-,17 - 20 June,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
181,ESH Creations,ESH Creations E39CHEQ High Gloss Semi Acoustic...,"₹5,397",7 days Replacement,"Saturday, 17 June",-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [34]:
Product.to_csv("Product.csv")

Q3.Write a python program to access the search bar and search button on images.google.com and scrape 10 
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 


In [27]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://images.google.com/" 
urls = []    
data = [] 

In [29]:
#Fruits images to download
search_item = ["fruits"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.XPATH,"//textarea[@class='gLFyf']")
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(search_item)
    # Clicking on search button
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,50)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])

   

In [30]:
#cars images to download
search_item = ["cars"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.XPATH,"//textarea[@class='gLFyf']")
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(search_item)
    # Clicking on search button
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,50)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])
    

In [31]:
search_item = ["Guitar"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.XPATH,"//textarea[@class='gLFyf']")
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(search_item)
    # Clicking on search button
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,50)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])

In [32]:
search_item = ["Machine Learning"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.XPATH,"//textarea[@class='gLFyf']")
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(search_item)
    # Clicking on search button
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,50)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])
    

In [33]:
search_item = ["cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.XPATH,"//textarea[@class='gLFyf']")
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(search_item)
    # Clicking on search button
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,50)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])
    

Q4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand 
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [239]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
url = "https://www.flipkart.com/"
driver.get(url)

In [240]:
search_bar = driver.find_element(By.XPATH,"//input[@class='_3704LK']")

# sending keys to search product
search_bar.send_keys("pixel 4A")

In [244]:
search_bar=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button')
search_bar.click()

In [245]:
page1_url = []
urls = driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']")
for url in urls:
    page1_url.append(url.get_attribute('href'))

In [246]:
len(page1_url)

5

In [247]:
Smartphones = ({})
Smartphones['Brand'] = []
Smartphones['Phone name'] = []
Smartphones['Colour'] = []
Smartphones['RAM'] = []
Smartphones['Storage(ROM)'] = []
Smartphones['Primary Camera'] = []
Smartphones['Secondary Camera'] = []
Smartphones['Display Size'] = []
Smartphones['Battery Capacity'] = []
Smartphones['Price'] = []
Smartphones['URL'] = []

In [248]:
for url in page1_url:
    driver.get(url)
    print("Scraping URL = ",url)
    Smartphones['URL'].append(url)
    time.sleep(2)
    
    
    #clicking on read more button to get more information
    try:
        read_more = driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']")
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    #scraping brand name of smartphone
    try:
        brand_tags = driver.find_element(By.XPATH,"//span[@class='B_NuCI']")
        Smartphones['Brand'].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    # scraping name of smartphones
    try:
        name_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][1]/table/tbody/tr[3]/td[2]/ul/li")
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
        
    #scraping colour of smartphone
    try:
        color_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]/ul/li")
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
        
    # scraping RAM data of smartphone
    try:
        ram_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
        
    #scraping ROM data of smartphones
    try:
        rom = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table[1]/tbody/tr[1]/td[2]/ul/li")
        Smartphones['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
    # scraping  Primary camera data of smartphone
    try:
        pri =driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['Primary Camera'].append(pri.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
    # scraping secondary camera data of smartphone
    try:
        sec = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[6]/td[1]")
        if sec != 'Secondary Camera' :
            if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[5]/td[1]").text == "Secondary Camera":
                sec_cam =driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[5]/td[2]/ul/li")
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[6]/td[2]/ul/li")
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
    
    #scraping display size data of smartphone
    try:
        disp = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/div")
        if disp.text != 'Display Features' : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/table[1]/tbody/tr[1]/td[2]/ul/li")
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
        
 # scraping the battery capacity of smartphone
    try:
        if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/div").text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/div").text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/table/tbody/tr/td[1]")
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/table/tbody/tr/td[2]/ul/li")
            elif driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/div").text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/table/tbody/tr/td[1]")
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/table/tbody/tr/td[2]/ul/li")
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/table/tbody/tr/td[1]")
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/table/tbody/tr/td[2]/ul/li")
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
    
    
    
    
    # scraping price of smartphone
    try:
        price_tags = driver.find_element(By.XPATH,"//div[@class='_30jeq3 _16Jk6d']")
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
          Smartphones['Price'].append('-')      

Scraping URL =  https://www.flipkart.com/google-pixel-6a-charcoal-128-gb/p/itme5ae89135d44e?pid=MOBGFKX5YUXD74Z3&lid=LSTMOBGFKX5YUXD74Z3MXA2OB&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=1ec7fd02-e91c-4be3-9be0-0ab6608af976.MOBGFKX5YUXD74Z3.SEARCH&ppt=hp&ppn=homepage&ssid=jltnpvyw2o0000001686393510326&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/google-pixel-6a-chalk-128-gb/p/itme5ae89135d44e?pid=MOBGFWEZ5SKU84Z8&lid=LSTMOBGFWEZ5SKU84Z8FXPB45&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=1ec7fd02-e91c-4be3-9be0-0ab6608af976.MOBGFWEZ5SKU84Z8.SEARCH&ppt=hp&ppn=homepage&ssid=jltnpvyw2o0000001686393510326&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/google-pixel-3a-just-black-64-gb/p/itmdb899b4e1bba7?pid=MOBGPAFV9PVXZCHD&lid=LSTMOBGPAFV9PVXZCHDAW1C4L&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=se

In [250]:
print(len(Smartphones['Brand']),
      len(Smartphones['Phone name']),
      len(Smartphones['Colour']),
      len(Smartphones['RAM']),
      len(Smartphones['Storage(ROM)']),
      len(Smartphones['Primary Camera']),len(Smartphones['Secondary Camera']),len(Smartphones['Display Size']),
      len(Smartphones['Battery Capacity']),len(Smartphones['Price']),len(Smartphones['URL']))

5 5 5 5 5 5 5 5 5 5 5


In [251]:
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,Google,Pixel 6a,Charcoal,6 GB,128 GB,12.2MP + 12MP,8MP Front Camera,15.6 cm (6.14 inch),4410 mAh,"₹27,999",https://www.flipkart.com/google-pixel-6a-charc...
1,Google,Pixel 6a,Chalk,6 GB,128 GB,12.2MP + 12MP,8MP Front Camera,15.6 cm (6.14 inch),4410 mAh,"₹27,999",https://www.flipkart.com/google-pixel-6a-chalk...
2,Google,Pixel 3A,Just Black,4 GB,64 GB,12.2MP Rear Camera,-,14.22 cm (5.6 inch),3000 mAh,"₹17,499",https://www.flipkart.com/google-pixel-3a-just-...
3,Google,Pixel 3,Not Pink,4 GB,128 GB,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,13.97 cm (5.5 inch),2915 mAh,"₹80,000",https://www.flipkart.com/google-pixel-3-not-pi...
4,Google,Pixel 3A,Purple-ish,4 GB,64 GB,12.2MP Rear Camera,-,14.22 cm (5.6 inch),3000 mAh,"₹17,899",https://www.flipkart.com/google-pixel-3a-purpl...


In [252]:
df.to_csv("smartphones.csv")

In [253]:
driver.close()

Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [62]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

In [63]:
url = 'https://www.google.co.in/maps'
driver.get(url)
time.sleep(2)

In [64]:
City = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.ID,'searchboxinput')
search_bar.click()
time.sleep(2)
search_bar.send_keys(City)
search_btn = driver.find_element(By.ID,"searchbox-searchbutton")
search_btn.click()

Enter City name that has to be searched : hyderabad


In [65]:
try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*)data',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))
except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/Hyderabad,+Telangana/@17.4120779,78.0783747,10z/data=!3m1!4b1!4m6!3m5!1s0x3bcb99daeaebd2c7:0xae93b78392bafbc2!8m2!3d17.385044!4d78.486671!16zL20vMDljNnc?entry=ttu
Latitude = 17.4120779, Longitude = 78.0783747


Q6 : Write a program to scrap all the available details of best gaming laptops from digit.in.

In [23]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

In [56]:
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [57]:
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(3)

In [58]:
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [59]:
laptop_name = driver.find_elements(By.XPATH,"//span[@class='datahreflink']/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    

try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


try:
    display = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']/table/tbody/tr[3]/td[3]")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass



try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']/table/tbody/tr[2]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass



try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass


try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass

try:
    graph = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


try:
    price = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]")
    for pri in price:
        Price.append(pri.text.replace('₹ ','Rs'))
except NoSuchElementException:
    pass

In [60]:
print(len(Laptop_Name),
len(Operating_sys),
len(Display),
len(Processor),
len(Memory),
len(Weight), 
len(Graph_proc),
len(Price))

7 7 7 7 7 7 7 7


In [61]:
import pandas as pd
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = Laptop_Name
Gaming_Laptop['Operating System'] =Operating_sys
Gaming_Laptop['Display'] = Display
Gaming_Laptop['Processor'] = Processor
Gaming_Laptop['Memory'] = Memory
Gaming_Laptop['Weight'] = Weight
Gaming_Laptop['Graphical Processor'] = Graph_proc
Gaming_Laptop['Price'] = Price
Gaming_Laptop

,Laptop Name,Operating System,Display,Processor,Memory,Weight,Graphical Processor,Price
0,HP OMEN 17 (2023),Windows 11 Home,"17.3″ (2560 x 1440) screen, 165 Hz refresh rate",13th Gen Intel Core i7-13700HX 16 core process...,16 GB DDR5GB RAM & 1 TB SSD,397.1 x 262 x 27 mm dimension & 2.78 kg weight,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,"Rs269,777"
1,MSI Titan GT77 12UHS,Windows 11 Home,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",12th Gen Intel Core i9-12900HX 16 core process...,64 GB DDR5GB RAM & 2 TB SSD,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...,"Rs499,990"
2,Lenovo Legion 5i Pro,Windows 11 Home,16″ (2560 x 1600) screen,12th Gen Intel Core i7-12700H 14 core processo...,32 GB DDR5GB RAM & 1 TB SSD,359.9 x 264.4 x 19.9 mm dimension & 3.6 kg weight,NVIDIA GeForce RTX 3070 Ti Graphics card,"Rs213,900"
3,ASUS ROG Strix Scar 18 2023,Windows 11 Home,"18″ (1920 x 1200) screen, 165 Hz refresh rate",13th Gen Intel Core i9-13980HX 24 core process...,32 GB DDR5GB RAM & 1 TB SSD,294 x 399 x 23 mm dimension & 3.1 kg weight,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,"Rs279,990"
4,Acer Predator Helios Neo 16,Windows 11 Home,"16″ (2560 x 1600) screen, 165 Hz refresh rate",13th Gen Intel Core i7-13700HX processor,16 GB DDR5GB RAM & 1 TB SSD,360 x 279 x 28 mm dimension & 2.6 kg weight,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,"Rs149,990"
5,ASUS ROG Zephyrus G14,Windows 11 Home,"14″ (1920 x 1200) screen, 144 Hz refresh rate",AMD Ryzen 9-6900HS 8 core processor with 4.9 G...,16 GB DDR5GB RAM & 1 TB SSD,312 x 227 x 19 mm dimension & 1.65 kg weight,8 GB DDR6 AMD Radeon RX 6700S Graphics card,"Rs156,990"
6,MSI Cyborg 15,Windows 11 Home,"15.6″ (1920 x 1080) screen, 144 Hz refresh rate",12th Gen Intel Core i7-12650H 10 core processo...,16 GB DDR5GB RAM & 1 TB SSD,& 1.98 kg weight,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,"Rs130,990"


Q7.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 

In [2]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(3)

#clicking on the explore button
button = driver.find_element(By.XPATH,"//div[@class='_69hVhdY4']")
button.click()
time.sleep(1)

#select billionaire  
bill = driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]")
bill.click()
time.sleep(1)

world_bill= driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/main/div/section/div[2]/div/div/div[1]/div/div[1]/div/div/a/h2")
world_bill.click()
time.sleep(1)

In [3]:
Rank = []
Person_Name = []
Net_worth  = []
Age = []
Citizenship = []
Source = []
Industry = []

while(True):
    
     # scraping the data of rank of the billionaires
    rank_tag = driver.find_elements(By.XPATH,"//div[@class='Table_rank___YBhk Table_dataCell__2QCve']")
    for i in rank_tag:
        rank_tag=i.text
        Rank.append(rank_tag)
        time.sleep(1)
        
 
    # scraping the data  of names of the billionaires
    name_tag = driver.find_elements(By.XPATH,"//div[2][@class='TableRow_cell__db-hv Table_cell__houv9']")
    for i in name_tag:
        name_tag=i.text
        Person_Name.append(name_tag)
        time.sleep(1)
    
    
    # scraping the data of age of the billionaires
    age_tag = driver.find_elements(By.XPATH,"//div[4][@class='TableRow_cell__db-hv Table_cell__houv9']")
    for i in age_tag:
        age_tag=i.text
        Age.append(age_tag)
        time.sleep(1)
    
    
    # scraping the data of citizenship of the billionaires
    cit_tag = driver.find_elements(By.XPATH,"//div[5][@class='TableRow_cell__db-hv Table_cell__houv9']") 
    for i in cit_tag:
        cit_tag=i.text
        Citizenship.append(cit_tag)
        time.sleep(1)
     
    
    # scraping the data of source of income of the billionaires
    sour_tag = driver.find_elements(By.XPATH,"//div[6][@class='TableRow_cell__db-hv Table_cell__houv9']")
    for i in sour_tag:
        sour_tag=i.text
        Source.append(sour_tag)
        time.sleep(1)
    
    
    # scraping data of industry of the billionaires
    ind_tag = driver.find_elements(By.XPATH,"//div[7][@class='TableRow_cell__db-hv Table_cell__houv9']")
    for i in ind_tag:
        ind_tag=i.text
        Industry.append(ind_tag)
        time.sleep(1)
    
    
    # scraping data of net_worth of billionaires
    net_tag = driver.find_elements(By.XPATH,"//div[3][@class='TableRow_cell__db-hv Table_cell__houv9']")
    for i in net_tag:
        net_tag=i.text
        Net_worth .append(net_tag)
        time.sleep(1)

    try:
        next_button = driver.find_element(By.XPATH,"//div[@class='Pagination_pagination__V7MBu']/button[2]")
        next_button.click()
    except:
        break
        


In [4]:
print(len(Rank),len(Person_Name),len(Age),len(Citizenship),len(Source),len(Industry),len(Net_worth))

2640 2640 2640 2640 2640 2640 2640


In [5]:
import pandas as pd
df=pd.DataFrame({'Rank':Rank,
                 'Person Name':Person_Name,
                 'Net Worth': Net_worth,
                 'Age':Age,
                 'Country':Citizenship,
                 'Source':Source,
                 'Industry':Industry})

In [6]:
df

,Rank,Person Name,Net Worth,Age,Country,Source,Industry
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,Fashion & Retail
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,$114 B,59,United States,Amazon,Technology
3,4,Larry Ellison,$107 B,78,United States,Oracle,Technology
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2635,2540,Yu Rong,$1 B,51,China,Health clinics,Healthcare
2636,2540,"Richard Yuengling, Jr.",$1 B,80,United States,Beer,Food & Beverage
2637,2540,Zhang Gongyun,$1 B,60,China,Tyre manufacturing machinery,Manufacturing
2638,2540,Zhang Guiping & family,$1 B,71,China,Real estate,Real Estate


In [7]:
df.to_csv("Billonaires.csv")

Q8 : Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [53]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [46]:
url = "https://www.youtube.com/"
search_item = ["natu natu song"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.XPATH,"//div[@class='ytd-searchbox-spt']/input")
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(search_item)
    search_btn = driver.find_element(By.ID,"search-icon-legacy")
    search_btn.click()
    time.sleep(2)
    # Clicking on search button
    search_button =driver.find_element(By.XPATH,"//yt-formatted-string[@class='style-scope ytd-video-renderer']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(1000):
        driver.execute_script("window.scrollBy(0,1000)")

In [47]:
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

# scrape comments
cm = driver.find_elements(By.ID,"content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)
time.sleep(4)


# scrape time when comment was posted
tm = driver.find_elements(By.XPATH,"//a[contains(text(),'ago')]")
for i in tm:
    Time.append(i.text)
    
for i in range(0,len(Time)):
    comment_time.append(Time[i])
time.sleep(4)


# scrape the comment likes
like = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])

In [48]:
print(len(comments),len(comment_time),len(No_of_Likes))

720 720 720


In [49]:
Youtube = pd.DataFrame({})
Youtube['Comment'] = comments[:500]
Youtube['Comment Time'] = comment_time[:500]
Youtube['Comment Upvotes'] = No_of_Likes[:500]
Youtube

,Comment,Comment Time,Comment Upvotes
0,Oscar winning moment. Proud moment for every i...,2 months ago (edited),160
1,"FULLY deserved the Oscar , as an Indian I’m ju...",2 months ago,200
2,The love he has for his friend always gets me....,4 weeks ago,240
3,This song is so ! I can't stop singing it! Th...,2 weeks ago,13
4,Proud moment for India this song won Oscar awa...,1 month ago,76
...,...,...,...
495,Very good,1 month ago,1
496,Seeing again after 1 year,4 weeks ago,
497,Lovely,5 days ago,1
498,I rewatched this scene so many times on Netfli...,9 months ago,422


In [50]:
Youtube.to_csv("Youtube Natu natu Comments.csv")

Q9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall 
reviews, privates from price, dorms from price, facilities and property description. 

In [2]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(3)

In [4]:
try:
        search_loc = driver.find_element(By.ID,'location-text-input-field')
        search_loc.send_keys("London")
        
except NoSuchElementException as e:
        time.sleep(5)

In [5]:
try:
    london_loc = driver.find_element(By.ID,'search-input-field')
    london_loc.send_keys("London")
except NoSuchElementException as e:
      time.sleep(5)

In [6]:
try:
    london = driver.find_element(By.XPATH,'//html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
    london.click()

except NoSuchElementException as e:
    time.sleep(5)

In [7]:
try:
    letsgo = driver.find_element(By.XPATH,'//div[@class="search-button"]/button')
# write Lonodn in search bar
    letsgo.click()
except ElementClickInterceptedException  as e:
      time.sleep(5)

In [8]:
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description = []
product_url= []

In [76]:
for i in driver.find_elements(By.XPATH,"//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    
    time.sleep(4)
    #fetching hostel name
    #for i in driver.find_elements(By.XPATH,"//div[@class='property']"):
    try:
            name = driver.find_elements(By.XPATH,"//h2[@class='title title-6']")
            for i in name:
                hostel_name.append(i.text)
    except NoSuchElementException:
               hostel_name.append('-')
    #fetching distance from city centre
    
    try:
            dist = driver.find_elements(By.XPATH,"//div[@class='subtitle body-3']/a/span[1]")
            for i in dist:
                distance.append(i.text)
    except NoSuchElementException:
               distance.append('-')
        
            #fetching facilities
    try:
            fac1 = driver.find_elements(By.XPATH,"//div[@class='has-wifi']")
            fac2 = driver.find_elements(By.XPATH,"//div[@class='has-sanitation']")
            for i in fac1:
                for j in fac2:
                    facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
                   facilities.append('-')
                

    for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):
    #fetch privates from price
        try:
            pvt_price = driver.find_element(By.XPATH,"//div[1][@class='prices-col']")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
            
    #fetching dorms from price
    
        try:
            dorms = driver.find_element(By.XPATH,"//div[2][@class='prices-col']")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')            
    
   
   
    #lets fetch url of each hostel
    p_url = driver.find_elements(By.XPATH,"//div[@class='prices-col']/a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

    for i in product_url:
        driver.get(i)
        time.sleep(3)
    #lets click on show more button for description
        try:
            driver.find_element(By.XPATH,"//button[@class='button primary small full-width']").click()
            time.sleep(3)
        except ElementClickInterceptedException as e1:
               time.sleep(3)
        except NoSuchElementException as e2:
             pass
   
     #fetching ratings
        try:
            rat = driver.find_element(By.XPATH,"//div[@class='score orange big' or @class='score gray big']")
            rating.append(rat.text)
        except NoSuchElementException:
            rating.append('-')
    #fetching total reviews
        
        try:
            rws = driver.find_element(By.XPATH,"//div[@class='reviews']")
            reviews.append(rws.text.replace('Total Reviews',''))
        except NoSuchElementException:
               reviews.append('-')
        #fetch overall review
        try:
            overall_rw = driver.find_element(By.XPATH,"//div[@class='keyword']/span")
            over_all.append(overall_rw.text)
        except NoSuchElementException:
            over_all.append('-') 



    #fetch property description 
    try:
        disc = driver.find_element(By.XPATH,"//div[@class='description-container']/div/div[2]")
        description.append(disc.text)
    except NoSuchElementException:
           over_all.append('-')

In [79]:
df = pd.DataFrame({})
df['Hostel_Name'] = hostel_name
df['Distance fron city centre'] = distance
df['Ratings'] = rating[:30]
df['Total_reviews'] = reviews[:30]
df['Overall Reviews'] = over_all[:30]
df['Privates from price'] = pvt_prices
df['Dorms from price'] = dorms_price
df['Facilities'] = facilities[:30]

In [80]:
df

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Facilities
0,Wombat's City Hostel London,Hostel - 3.6km from city centre,9.0,14896,Superb,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
1,St Christopher's Village,Hostel - 1.8km from city centre,7.9,12155,Very Good,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
2,Urbany Hostel London,Hostel - 5.4km from city centre,9.4,780,Superb,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
3,NX London Hostel,Hostel - 6.1km from city centre,8.3,1885,Fabulous,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
4,Generator London,Hostel - 3km from city centre,7.5,7500,Very Good,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
5,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,7.5,4957,Very Good,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
6,Phoenix Hostel,Hostel - 4.2km from city centre,7.6,4126,Very Good,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
7,Safestay London Kensington Holland Park,Hostel - 5.8km from city centre,7.1,1528,Very Good,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
8,No.8 Seven Sisters,Hostel - 9km from city centre,5.7,4010,Rating,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
9,Queen Elizabeth Chelsea,Hostel - 5.7km from city centre,7.4,3480,Very Good,-,-,"Free WiFi, Follows Covid-19 sanitation guidance"
